In [1]:
import os
import re
os.chdir('../')

In [2]:
import json
import time
import requests
from datetime import datetime

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
  

/Users/manish.sahu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from src.general_utils import util

In [281]:
location_list = [
    'bengaluru',
    # 'national-capital-region-ncr',
    'mumbai',
    'chennai',
    'kolkata',
    'bhopal',
    'indore',
    'chandigarh',
    'ahmedabad',
    'hyderabad',
    'pune',
    'kochi',
    'varanasi',
    'jaipur',
    'bhubaneswar',
]

In [5]:
class GetMovieDetails:
    def __init__(self, path_movie_url, movie_name, date, location, if_debug=False):
        
        self.path_movie_url = path_movie_url   
        self.movie_name = movie_name 
        self.date = date
        self.location = location
        self.formatted_time = self._get_current_time()
        self.if_debug = if_debug

    def _get_current_time(self):
        # Get the current time in seconds since the epoch
        current_time_seconds = time.time()

        # Convert seconds since the epoch to a time structure
        time_struct = time.localtime(current_time_seconds)

        # Format the time in 24-hour format (HH:MM:SS)
        formatted_time = time.strftime("%H:%M", time_struct)
        return formatted_time
    
    def _get_header(self):
        headers = {
                'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
                'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': "Linux",
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                }
        return headers
        
    
    def _get_chrome_driver(self):
        # driver = webdriver.Chrome(ChromeDriverManager().install())
        driver = webdriver.Chrome()
        driver.get(self.path_movie_url)
        return driver

    
    def convert_to_json(self, data_categories:str):
        return json.loads(data_categories)

    def get_price(self, data_json):
        return data_json.get('price')

    def get_desc(self, data_json):
        return data_json.get('desc')

    def get_availability_class(self, data_json):
        return data_json.get('availabilityClass')

    def get_availStatus(self, data_json):
        return data_json.get('availStatus')

    def get_processed_df(self):
        driver = self._get_chrome_driver()
        
        if self.if_debug : print(driver)
        theatre_element_list = driver.find_elements(By.CLASS_NAME, 'list')
        if self.if_debug : print(f'Number of theatres : {len(theatre_element_list)}')

        data = []

        for i, a in tqdm(enumerate(theatre_element_list)):
            try:
                titles = a.find_element(By.CLASS_NAME, '__title')
                listing_info = a.find_elements(By.CLASS_NAME, 'listing-info')
                showtimes = a.find_elements(By.CLASS_NAME, 'showtime-pill')

                for s in showtimes:

                    data_categories = self.convert_to_json(s.get_attribute('data-categories'))
                    timings = s.get_attribute('data-date-time')

                    temp_data = {
                        'cinema_title' : titles.text.split('\nINFO')[0],
                        'timings': timings,
                        'latitude': a.get_attribute('data-lat'),
                        'longtitude' : a.get_attribute('data-lng'),

                        'price': self.get_price(data_categories),
                        'description': self.get_desc(data_categories),

                        'availability_class': self.get_availability_class(data_categories),
                        'avail_status': self.get_availStatus(data_categories),
                        'movie_name' : self.movie_name,
                        'date' : self.date,
                        'location' : self.location,
                        'checked_time' : self.formatted_time,
                    }
                    
                    data.append(temp_data)

                # b.find_element('showtime-pill')
            except:
                if self.if_debug : print(i)

        
        data_df = pd.DataFrame(data)

        return data_df
        

In [6]:
class GetMoviesList:
    def __init__(self, path_movie_list_url, location, date):
        self.date = date
        self.location = location
        self.path_movie_list_url = path_movie_list_url    
        self.movie_ahref_string = f'https://in.bookmyshow.com/{location}/movies'

    def _get_header(self):
        headers = {
                'Referer': 'https://in.bookmyshow.com/explore/movies-bengaluru?languages=hindi',
                'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': "Linux",
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                }
        return headers

    def _get_chrome_driver(self):
        # driver = webdriver.Chrome(ChromeDriverManager().install())
        driver = webdriver.Chrome()
        driver.get(self.path_movie_list_url)
        
        # Scroll to buttom slowly to load all movies
        driver.execute_async_script(
            """
        count = 400;
        let callback = arguments[arguments.length - 1];
        t = setTimeout(function scrolldown(){
            console.log(count, t);
            window.scrollTo(0, count);
            if(count < (document.body.scrollHeight || document.documentElement.scrollHeight)){
              count+= 400;
              t = setTimeout(scrolldown, 1000);
            }else{
              callback((document.body.scrollHeight || document.documentElement.scrollHeight));
            }
        }, 1000);"""
        )
        return driver
    
    def parse_drive_to_soup(self, driver):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        return soup
    
    def get_all_current_movie_list(self, soup):
        
        movie_soup_element_dict = {}
        
        movie_tags_list = [a for a in soup.find_all('a', href=True) if self.movie_ahref_string in a['href']]
        
        # Extract movie names from the URLs
        movie_names = []
        for link in movie_tags_list:
            # Extract the movie name from the URL
            movie_name_match = re.search(r'.*/movies/(.*)/ET.*', link['href'])
            if movie_name_match:
                movie_name = movie_name_match.group(1)
                movie_names.append(movie_name)

                movie_url = link['href']
                movie_url_basename = movie_url.split('/')[-1]
                
                movie_booking_url = f'https://in.bookmyshow.com/buytickets/{movie_name}-{self.location}/movie-bang-{movie_url_basename}-MT/{self.date}'

                movie_soup_element_dict[movie_name] = {
                    # 'element': link,
                    'url': movie_url,
                    'name': movie_name,
                    'booking_url' : movie_booking_url,
                    'date': self.date,
                    'location' : self.location,
                }
            
        # # Print the movie names
        # for movie_name in movie_names:
        #     print("Movie Name:", movie_name)

        
        print(f'Number of movies running: {len(movie_names)}')
        return movie_soup_element_dict
        
    def generate(self):
        driver = self._get_chrome_driver()
        soup = self.parse_drive_to_soup(driver)
        movie_soup_element_dict  = self.get_all_current_movie_list(soup)
        
        return movie_soup_element_dict

In [282]:
for location in location_list:
    print(f'Processing location: {location}')
    # location = 'national-capital-region-ncr'
    # Get the current date
    current_date = datetime.now()
    # Format the date in YYYYMMDD format
    date = current_date.strftime("%Y%m%d")
    # Get the current time in seconds since the epoch
    current_time_seconds = time.time()

    # Convert seconds since the epoch to a time structure
    time_struct = time.localtime(current_time_seconds)

    # Format the time in 24-hour format (HH:MM:SS)
    formatted_time = time.strftime("%H:%M", time_struct)

    path_movie_list_url = f'https://in.bookmyshow.com/explore/movies-{location}'
    path_output_dir = 'data/'

    #######################
    path_output_data_csv = os.path.join(path_output_dir, f'{location}/{date}/data_{location}_{date}.csv')
    path_output_data_json = os.path.join(path_output_dir, f'{location}/{date}/data_{location}_{date}.json')

    print(f'path_output_data_csv: {path_output_data_csv}')
    
    #######################
    movie_list_instance = GetMoviesList(path_movie_list_url=path_movie_list_url, location=location, date=date)
    movie_soup_element_dict = movie_list_instance.generate()
    
    util.check_dir(os.path.dirname(path_output_data_json))
    util.save_json(movie_soup_element_dict, path_output=path_output_data_json)
    
    
    #######################
    data_movie_list = []

    for movie_name in tqdm(movie_soup_element_dict):
        path_movie_url = movie_soup_element_dict[movie_name]['booking_url']

        movie_details = GetMovieDetails(path_movie_url=path_movie_url,
                                        location=location, date=date,
                                        movie_name=movie_name,
                                        
                                        )
        movie_detail_df = movie_details.get_processed_df()
        data_movie_list.append(movie_detail_df)


    data_movie_list_pd = pd.concat(data_movie_list)
    
    
    if os.path.exists(path_output_data_csv):
        data_movie_list_pd.to_csv(path_output_data_csv, index=False, mode='a', header=False)
    else:
        data_movie_list_pd.to_csv(path_output_data_csv, index=False, mode='a', header=True)

0it [00:00, ?it/s]
0it [00:00, ?it/s]/16 [00:52<05:44, 24.60s/it]
0it [00:00, ?it/s]/16 [01:12<04:52, 22.49s/it]
0it [00:00, ?it/s]/16 [01:26<03:49, 19.14s/it]
 31%|███▏      | 5/16 [01:48<03:58, 21.65s/it]


KeyboardInterrupt: 

In [276]:
# movie_list_instance = GetMoviesList(path_movie_list_url=path_movie_list_url, location=location, date=date)
# movie_soup_element_dict = movie_list_instance.generate()

Number of movies running: 16


In [278]:
# util.check_dir(os.path.dirname(path_output_data_json))
# util.save_json(movie_soup_element_dict, path_output=path_output_data_json)

In [279]:
# data_movie_list = []

# for movie_name in tqdm(movie_soup_element_dict):
#     path_movie_url = movie_soup_element_dict[movie_name]['booking_url']

#     movie_details = GetMovieDetails(path_movie_url=path_movie_url,
#                                     location=location, date=date,
#                                     movie_name=movie_name,
                                    
#                                     )
#     movie_detail_df = movie_details.get_processed_df()
#     data_movie_list.append(movie_detail_df)


# data_movie_list_pd = pd.concat(data_movie_list)

13it [00:00, 52.58it/s]00:00<?, ?it/s]
50it [00:02, 17.47it/s]00:19<04:53, 19.58s/it]
19it [00:00, 37.25it/s]00:40<04:42, 20.15s/it]
13it [00:00, 45.95it/s]00:55<03:50, 17.76s/it]
35it [00:01, 21.70it/s]01:13<03:36, 18.04s/it]
0it [00:00, ?it/s]/16 [01:32<03:22, 18.43s/it]
12it [00:00, 50.33it/s]01:50<03:02, 18.29s/it]
10it [00:00, 73.46it/s]02:05<02:35, 17.23s/it]
12it [00:00, 51.82it/s]02:23<02:19, 17.44s/it]
14it [00:00, 45.66it/s]02:45<02:12, 18.89s/it]
11it [00:00, 64.49it/s][03:01<01:48, 18.06s/it]
0it [00:00, ?it/s]1/16 [03:19<01:29, 17.81s/it]
0it [00:00, ?it/s]2/16 [03:37<01:12, 18.09s/it]
0it [00:00, ?it/s]3/16 [03:52<00:51, 17.09s/it]
57it [00:03, 16.92it/s][04:03<00:30, 15.24s/it]
0it [00:00, ?it/s]5/16 [04:27<00:17, 17.86s/it]
100%|██████████| 16/16 [04:45<00:00, 17.82s/it]


In [146]:
# if os.path.exists(path_output_data_csv):
#     data_movie_list_pd.to_csv(path_output_data_csv, index=False, mode='a', header=False)
# else:
#     data_movie_list_pd.to_csv(path_output_data_csv, index=False, mode='a', header=True)

In [ ]:
Filters - 
1. location
2. Movie
3. Date 
4. Time of day

Stats - 
1. Number of Movies. 
2. Number of Theaters 
3. Occupancy Chart 

Times Series - 
1. Number of Shows per day
2. Price of tickets per day
3. Occupancy rate per day


In [237]:
from src.data_utils import data_loader
import importlib

importlib.reload(data_loader)

<module 'src.data_utils.data_loader' from '/Users/manish.sahu/Downloads/tiler/scrap-bookmyshow/src/data_utils/data_loader.py'>

In [238]:
city = 'bengaluru'
path_data_dir = '/Users/manish.sahu/Downloads/tiler/scrap-bookmyshow/data'

movie_data_loader = data_loader.MovieDataLoder(path_data_dir)
city_df = movie_data_loader.get_city_data(city=city)

In [12]:
# # for holding the resultant list
# element_list = []
  
# for page in range(1, 3, 1):
    
#     page_url = "https://webscraper.io/test-sites/e-commerce/static/computers/laptops?page=" + str(page)
#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver.get(page_url)
#     title = driver.find_elements_by_class_name("title")
#     price = driver.find_elements_by_class_name("price")
#     description = driver.find_elements_by_class_name("description")
#     rating = driver.find_elements_by_class_name("ratings")
  
#     for i in range(len(title)):
#         element_list.append([title[i].text, price[i].text, description[i].text, rating[i].text])
  
# print(element_list)
  
# #closing the driver
# driver.close()